In [7]:
words = open('names.txt', 'r').read().splitlines()

In [8]:
import torch

In [9]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0

In [10]:
itos = {i:s for s, i in stoi.items()}

In [11]:
# create the training set of bigrams (x, y)

xs, ys = [], []
num_bigrams = 0

for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        xs.append(ix1)
        ys.append(ix2)
        num_bigrams += 1

xs = torch.tensor(xs)
ys = torch.tensor(ys)

In [12]:
import torch.nn.functional as F # type: ignore

# initialize the network
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g, requires_grad=True)


In [16]:
for k in range(100):
    # forward pass
    xenc = F.one_hot(xs, num_classes=27).float() # type: ignore
    logits = xenc @ W  # predict log-counts
    counts = logits.exp()  # counts, equivalent to N
    probs = counts / counts.sum(1, keepdim=True)  # probabilities for next character
    loss = -probs[torch.arange(num_bigrams), ys].log().mean()  # + 0.01 * (W**2).mean()
    print(loss.item())

    # backward pass
    W.grad = None
    loss.backward()

    # update
    W.data += -1 * W.grad # type: ignore


3.184269428253174
3.1808712482452393
3.1775028705596924
3.1741647720336914
3.1708550453186035
3.167574405670166
3.1643218994140625
3.161098003387451
3.1579015254974365
3.1547322273254395
3.151589870452881
3.148474931716919
3.1453857421875
3.142322540283203
3.1392853260040283
3.13627290725708
3.1332857608795166
3.130323886871338
3.1273860931396484
3.12447190284729
3.121581792831421
3.1187150478363037
3.1158714294433594
3.113051176071167
3.1102535724639893
3.107478380203247
3.104724884033203
3.1019933223724365
3.099283456802368
3.096595048904419
3.0939269065856934
3.091279983520508
3.088653802871704
3.086047649383545
3.0834615230560303
3.0808956623077393
3.0783488750457764
3.0758213996887207
3.0733134746551514
3.070824146270752
3.0683536529541016
3.065901517868042
3.063467502593994
3.061051607131958
3.0586540699005127
3.0562734603881836
3.053910732269287
3.051565647125244
3.04923677444458
3.0469253063201904
3.0446300506591797
3.0423519611358643
3.0400900840759277
3.037843942642212
3.0356

In [17]:
for i in range(5):
    out = []
    ix = 0

    while True:
        xenc = F.one_hot(torch.tensor([ix]), num_classes=27).float()
        logits = xenc @ W
        counts = logits.exp()
        p = counts / counts.sum(1, keepdim=True)
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix]) # type: ignore
        if ix == 0:
            break
    
    print(''.join(out))


dayti.
kotjasihqzqrzw.
eie.
ke.
zw.
